In [2]:
import pandas as pd 
import requests as req
import json
import re

#%pip install shapely
#%pip install geopandas
from shapely.geometry import Point
import geopandas as gpd
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import folium
#%pip install python-Levenshtein
import folium
import polyline
import webbrowser

**ESTACIONES API**

In [3]:
with open('../contraseñas.txt', 'r') as file:
    # Leer la contraseña
    contraseña = file.read().strip().split('\n')


In [4]:
#connect to the EMT de madrid API to obtain my accessToken
url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'

headers = {
    'email': contraseña[0],
    'password': contraseña[1]
    }

response = req.request('GET', url, headers=headers)
response.json()

{'code': '01',
 'description': 'Token extend  into control-cache Data recovered  OK, (lapsed: 155 millsecs)',
 'datetime': '2024-03-16T09:53:14.744702',
 'data': [{'nameApp': 'OPENAPI MobilityLabs',
   'levelApp': 0,
   'updatedAt': '2024-03-05T18:29:05.2530000',
   'userName': 'veronicagrim@hotmail.com',
   'lastUpdate': {'$date': 1710579103586},
   'idUser': '2c4e1717-b9d6-4bcf-b026-38157bb8f83b',
   'priv': 'U',
   'tokenSecExpiration': 86399,
   'email': 'veronicagrim@hotmail.com',
   'tokenDteExpiration': {'$date': 1710669102586},
   'flagAdvise': True,
   'accessToken': 'f80fe3d1-a76b-452b-bc7e-21302f7228da',
   'apiCounter': {'current': 4,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs

In [5]:
#get the accessToken
accessToken=response.json()['data'][0]['accessToken']

In [6]:
#connect to the EMT de madrid API to obtain data from all bicimad stations.
url='https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'

headers = {
    'accessToken': accessToken,
    }

response = req.request("GET", url, headers=headers)
estaciones=response.json()

In [7]:
#convert json to data frame
estaciones=estaciones['data']
estaciones_api=pd.DataFrame(estaciones)


In [8]:
#clean name and separate latitude and longitude
names=[f['name'].split('-')[1].strip() for f in estaciones]
coordenadas=[[f['geometry']['coordinates'] for f in estaciones]]
coordenadas

estaciones_api['name']=names
estaciones_api['geometry']=coordenadas[0]  


#separate latitude and longitude
def dividir_lista(row):
    return pd.Series(row['geometry'])
estaciones_api[['longitude', 'latitude']] = estaciones_api.apply(dividir_lista, axis=1)
estaciones_api=estaciones_api.drop(columns=['geometry'])
estaciones_api

,activate,address,dock_bikes,free_bases,geofenced_capacity,id,light,name,no_available,number,reservations_count,total_bases,virtualDelete,longitude,latitude
0,1,"Calle Miguel Moya nº 1,",0,0,20,1406,3,Metro Callao,1,2,0,27,False,-3.705690,40.420400
1,1,"Plaza Conde Surchill, 4 ,",11,8,20,1407,2,Plaza Conde Suchil,0,3,0,19,False,-3.707254,40.430322
2,1,"Calle Fuencarral nº 106,",24,3,20,1409,1,Fuencarral,0,5,0,27,False,-3.702135,40.428521
3,1,"Calle Hortaleza nº 63,",2,17,20,1410,0,Colegio Arquitectos,0,6,0,19,False,-3.698447,40.424148
4,1,"Calle Hortaleza nº 75,",7,12,20,1411,2,Hortaleza,0,7,0,19,False,-3.697771,40.425191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,1,"Calle de las Delicias, 41 ,",11,8,20,2117,2,Delicias,0,273,0,19,False,-3.690918,40.403819
607,1,"Calle Camilo José Cela, 27,",6,17,20,1647,0,Camilo José Cela,0,241,0,23,False,-3.667790,40.437800
608,1,"Ronda de Atocha nº 34,",23,4,20,1453,1,Casa Encendida,0,49,0,27,False,-3.699277,40.406053
609,1,"Alcala-Ventas ,",18,5,20,1593,1,Alejandro González,0,187,0,23,False,-3.663825,40.430133


**ESTACIONES CSV**

In [9]:
estaciones_csv = pd.read_csv('../data/processed/estaciones.csv')
estaciones_csv

,name_x,address_x,number,distancia,name_y,address_y,activate,no_available,total_bases,dock_bikes,free_bases,longitude,latitude
0,Metro Callao,"Calle Miguel Moya nº 1,",2,101,Metro Callao,"Calle Miguel Moya nº 1,",1,1,27,0,0,-3.705690,40.420400
1,Metro Callao,"Calle Miguel Moya nº 1,",22,101,Jacometrezo,"Calle Jacometrezo nº 3,",1,1,3,0,0,-3.706538,40.420078
2,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",3,241,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",1,0,19,11,8,-3.707254,40.430322
3,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",12,241,San Hermenegildo,"Calle San Bernardo nº 85 - 87,",1,0,27,22,5,-3.706141,40.428467
4,Fuencarral,"Calle Fuencarral nº 106,",5,42,Fuencarral,"Calle Fuencarral nº 106,",1,0,27,23,4,-3.702135,40.428521
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,Casa Encendida,"Ronda de Atocha nº 34,",53,300,Plaza de Lavapiés,"Plazuela de Ana Diosdado,",1,0,24,14,10,-3.700754,40.408309
1218,Alejandro González,"Alcala-Ventas ,",187,324,Alejandro González,"Alcala-Ventas ,",1,0,23,18,5,-3.663825,40.430133
1219,Alejandro González,"Alcala-Ventas ,",193,324,Plaza de las Ventas,"Avenida De los Toreros, 49 ,",1,0,27,19,8,-3.665555,40.432472
1220,"Andaluces, 2","Calle de los Andaluces, 2,",427,344,"Andaluces, 2","Calle de los Andaluces, 2,",1,0,24,20,4,-3.641483,40.391424


In [10]:
estaciones = pd.merge(estaciones_csv,estaciones_api, on='number', how='left')
estaciones


,name_x,address_x,number,distancia,name_y,address_y,activate_x,no_available_x,total_bases_x,dock_bikes_x,...,geofenced_capacity,id,light,name,no_available_y,reservations_count,total_bases_y,virtualDelete,longitude_y,latitude_y
0,Metro Callao,"Calle Miguel Moya nº 1,",2,101,Metro Callao,"Calle Miguel Moya nº 1,",1,1,27,0,...,20,1406,3,Metro Callao,1,0,27,False,-3.705690,40.420400
1,Metro Callao,"Calle Miguel Moya nº 1,",22,101,Jacometrezo,"Calle Jacometrezo nº 3,",1,1,3,0,...,0,1425,3,Jacometrezo,1,0,3,False,-3.706538,40.420078
2,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",3,241,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",1,0,19,11,...,20,1407,2,Plaza Conde Suchil,0,0,19,False,-3.707254,40.430322
3,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",12,241,San Hermenegildo,"Calle San Bernardo nº 85 - 87,",1,0,27,22,...,20,1416,1,San Hermenegildo,0,0,27,False,-3.706141,40.428467
4,Fuencarral,"Calle Fuencarral nº 106,",5,42,Fuencarral,"Calle Fuencarral nº 106,",1,0,27,23,...,20,1409,1,Fuencarral,0,0,27,False,-3.702135,40.428521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,Casa Encendida,"Ronda de Atocha nº 34,",53,300,Plaza de Lavapiés,"Plazuela de Ana Diosdado,",1,0,24,14,...,20,1457,2,Plaza de Lavapiés,0,0,24,False,-3.700754,40.408309
1218,Alejandro González,"Alcala-Ventas ,",187,324,Alejandro González,"Alcala-Ventas ,",1,0,23,18,...,20,1593,1,Alejandro González,0,0,23,False,-3.663825,40.430133
1219,Alejandro González,"Alcala-Ventas ,",193,324,Plaza de las Ventas,"Avenida De los Toreros, 49 ,",1,0,27,19,...,20,1599,1,Plaza de las Ventas,0,0,27,False,-3.665555,40.432472
1220,"Andaluces, 2","Calle de los Andaluces, 2,",427,344,"Andaluces, 2","Calle de los Andaluces, 2,",1,0,24,20,...,20,2031,1,"Andaluces, 2",0,0,24,False,-3.641483,40.391424


**EMBAJADAS**

In [11]:
#read the csv created from embassies
embajadas = pd.read_csv('../data/processed/embajadas.csv')
embajadas

,title,street-address,latitude,longitude,number,type_of_place,distance
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,consulado,311.900114
1,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,consulado,319.381126
2,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,consulado,380.025843
3,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,consulado,255.765517
4,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,consulado,312.602762
...,...,...,...,...,...,...,...
466,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,embajada,422.553954
467,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,embajada,537.184597
468,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,embajada,179.924919
469,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,embajada,291.887270


In [12]:
#add data to the embassy table of the station api table.
result = pd.merge(embajadas, estaciones_api, on='number', how='left')
result

,title,street-address,latitude_x,longitude_x,number,type_of_place,distance,activate,address,dock_bikes,...,geofenced_capacity,id,light,name,no_available,reservations_count,total_bases,virtualDelete,longitude_y,latitude_y
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,consulado,311.900114,1,"Plaza de Alonso Martínez nº 6,",14,...,20,1412,2,Metro Alonso Martínez,0,0,24,False,-3.695440,40.427868
1,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,consulado,319.381126,1,"Calle Marqués de la Ensenada (junto a edif, Ce...",14,...,0,1414,2,Marqués de la Ensenada,0,0,23,False,-3.692104,40.425403
2,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,consulado,380.025843,1,"Paseo de la Castellana nº 4,",4,...,20,1507,0,Castellana,0,0,23,False,-3.689580,40.426770
3,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,consulado,255.765517,1,"Calle Serrano nº 210,",5,...,20,1554,0,Serrano 210,0,0,23,False,-3.681397,40.451117
4,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,consulado,312.602762,1,"Paseo de la Habana nº 63,",10,...,20,1565,2,Paseo de la Habana 63,0,0,23,False,-3.683593,40.454385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,embajada,422.553954,1,"Calle Mesena, 102 ,",6,...,20,2116,0,"Mesena, 102",0,0,24,False,-3.663406,40.467724
467,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,embajada,537.184597,1,"Calle Santa María Magdalena, 15 ,",15,...,20,1969,2,Santa María Magdalena,0,0,23,False,-3.670600,40.463100
468,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,embajada,179.924919,1,"Paseo de la Castellana nº 43,",7,...,20,1547,2,Plaza de Emilio Castelar,0,0,12,False,-3.689392,40.435405
469,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,embajada,291.887270,1,"Paseo de la Castellana nº 42,",14,...,20,1548,2,Bajo elevado de Eduardo Dato,0,0,23,False,-3.687915,40.433409


**FUNCIONES DE LLAMADA**

In [21]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

#functions to find the source and destination in the table they are in and return the data you want
coordenadas=[]
def buscar_origen(type_of_place_origen, origen):
    lat_start, long_start= 0, 0
    
    if type_of_place_origen.lower() == 'embajada' or type_of_place_origen.lower() == 'consulado':
        if type_of_place_origen.lower() == 'embajada':
            filtered_results = result[result['type_of_place'] == 'embajada']
        elif type_of_place_origen.lower() == 'consulado':
            filtered_results = result[result['type_of_place'] == 'consulado']
       
        posibles_coincidencias = process.extract(origen, filtered_results['title'], scorer=fuzz.WRatio, limit=3)
    
        ruta_data_origen = filtered_results[filtered_results['title'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_origen = ruta_data_origen[(ruta_data_origen['no_available'] == 0) & (ruta_data_origen['dock_bikes'] > 0)]
        lat_start = ruta_data_origen['latitude_y'].iloc[0]
        long_start = ruta_data_origen['longitude_y'].iloc[0]

        if len(ruta_data_origen) == 0:
            print(f"No se encontraron datos para el origen '{origen}' que cumplan con las condiciones")
        
        coordenadas.append((lat_start, long_start)) 
        mensaje = f'HOLA, estás en la/el {ruta_data_origen["title"].iloc[0].upper()} ({ruta_data_origen["street-address"].iloc[0].title()}). '
        mensaje += f' La estación de Bicimad más cercana es {ruta_data_origen["name"].iloc[0].upper()}({ruta_data_origen["address"].iloc[0].split(" ,")[0]}) '
        mensaje += f'a {round(ruta_data_origen["distance"].iloc[0])} metros de distancia y con {ruta_data_origen["dock_bikes"].iloc[0]} bicis libres.'
        return mensaje
    
    elif type_of_place_origen.lower() == 'estacion':
        posibles_coincidencias = process.extract(origen, estaciones['name_x'], scorer=fuzz.WRatio, limit=1)

        ruta_data_origen = estaciones[estaciones['name_x'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_origen = ruta_data_origen[(ruta_data_origen['no_available_y'] == 0) & (ruta_data_origen['dock_bikes_y'] > 0)]

        if len(ruta_data_origen) == 0:
            print(f"No se encontraron datos para el origen '{origen}' que cumplan con las condiciones")
        else:
            lat_start = ruta_data_origen['latitude_y'].iloc[0]
            long_start = ruta_data_origen['longitude_y'].iloc[0]
            
            coordenadas.append((lat_start, long_start))
            
            numeros_origen = ruta_data_origen['number'].tolist()
            cercanas = result[result['number'].isin(numeros_origen)]
            
            if not cercanas.empty:
                embajadas_cercanas = cercanas[['title', 'distance']].values.tolist()
                mensaje_cercanas = "Embajadas/Consulados cercanos:\n"
                for i, (embajada, distancia) in enumerate(embajadas_cercanas, 1):
                    mensaje_cercanas += f"{i}. {embajada} - {round(distancia)} metros\n"
            else:
                pass
           
            
    mensaje = f'HOLA, estás en la estación de Bicimad {ruta_data_origen["name_x"].iloc[0].upper()} ({ruta_data_origen["address_x"].iloc[0].split(" ,")[0].title()}), con {ruta_data_origen["dock_bikes_y"].iloc[0]} bicis libres.'   
    print (mensaje)
    
    return (mensaje_cercanas)

def buscar_destino(type_of_place_destino, destino):
    lat_finish, long_finish = 0, 0
    mensaje_cercanas = ""
    
    if type_of_place_destino.lower() == 'embajada' or type_of_place_destino.lower() == 'consulado':
        if type_of_place_destino.lower() == 'embajada':
            filtered_results = result[result['type_of_place'] == 'embajada']
        elif type_of_place_destino.lower() == 'consulado':
            filtered_results = result[result['type_of_place'] == 'consulado']
       
        posibles_coincidencias = process.extract(destino, filtered_results['title'], scorer=fuzz.WRatio, limit=3)
    
        ruta_data_destino = filtered_results[filtered_results['title'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_destino = ruta_data_destino[(ruta_data_destino['no_available'] == 0) & (ruta_data_destino['free_bases'] > 0)]
        
        if len(ruta_data_destino) == 0:
            print(f"No se encontraron datos para el destino '{destino}' que cumplan con las condiciones")
        else:
            lat_finish = ruta_data_destino['latitude_y'].iloc[0]
            long_finish = ruta_data_destino['longitude_y'].iloc[0]
            
            coordenadas.append((lat_finish, long_finish))
            
        mensaje = f'Tu destino es el siguiente consulado/embajada {ruta_data_destino["title"].iloc[0].upper()} ({ruta_data_destino["street-address"].iloc[0].title()}), con {ruta_data_destino["free_bases"].iloc[0]} aparcameintos libres.'
        return (mensaje)    
    
    elif type_of_place_destino.lower() == 'estacion':
        posibles_coincidencias = process.extract(destino, estaciones['name_x'], scorer=fuzz.WRatio, limit=1)

        ruta_data_destino = estaciones[estaciones['name_x'].isin([x[0] for x in posibles_coincidencias])]
        ruta_data_destino = ruta_data_destino[(ruta_data_destino['no_available_y'] == 0) & (ruta_data_destino['free_bases_y'] > 0)]
        
        if len(ruta_data_destino) == 0:
            print(f"No se encontraron datos para el destino '{destino}' que cumplan con las condiciones")
        else:
            lat_finish = ruta_data_destino['latitude_y'].iloc[0]
            long_finish = ruta_data_destino['longitude_y'].iloc[0]
            
            coordenadas.append((lat_finish, long_finish))
            
            numeros_destino = ruta_data_destino['number'].tolist()
            cercanas = result[result['number'].isin(numeros_destino)]
            
            if not cercanas.empty:
                embajadas_cercanas = cercanas[['title', 'distance']].values.tolist()
                mensaje_cercanas = "Embajadas/Consulados cercanos:\n"
                for i, (embajada, distancia) in enumerate(embajadas_cercanas, 1):
                    mensaje_cercanas += f"{i}. {embajada} - {round(distancia)} metros\n"
            else:
                pass
        
    mensaje = f'Tu destino es la estación de Bicimad {ruta_data_destino["name_x"].iloc[0].upper()} ({ruta_data_destino["address_x"].iloc[0].split(" ,")[0].title()}), con {ruta_data_destino["free_bases_y"].iloc[0]} aparcamientos libres.'
    print (mensaje)
    return (mensaje_cercanas)

In [22]:
type_of_place_origen= 'estacion'
origen= 'zubano'
type_of_place_destino= 'estacion'
destino= 'zurbano'

print(buscar_origen(type_of_place_origen, origen))
print ( )
print(buscar_destino(type_of_place_destino, destino))
print ( )
print(f'La distancia total en bici serán {round(distance_meters(coordenadas[0][0],coordenadas[0][1],coordenadas[1][0],coordenadas[1][1])[0])} metros.')

HOLA, estás en la estación de Bicimad ZURBANO (Calle De Zurbano, 90), con 8 bicis libres.
Embajadas/Consulados cercanos:
1. Consulado de Chile - 524 metros
2. Consulado de Costa Rica - 463 metros
3. Embajada de Costa Rica - 463 metros
4. Embajada de Eslovaquia - 355 metros
5. Embajada de Eslovenia - 416 metros
6. Embajada de Japón - 461 metros
7. Embajada de Jordania - 362 metros
8. Embajada de Nueva Zelanda - 318 metros
9. Embajada de Pakistán - 269 metros
10. Embajada de Perú - 246 metros
11. Embajada de Turquía - 524 metros
12. Embajada del Nepal - 468 metros


Tu destino es la estación de Bicimad ZURBANO (Calle De Zurbano, 90), con 15 aparcamientos libres.
Embajadas/Consulados cercanos:
1. Consulado de Chile - 524 metros
2. Consulado de Costa Rica - 463 metros
3. Embajada de Costa Rica - 463 metros
4. Embajada de Eslovaquia - 355 metros
5. Embajada de Eslovenia - 416 metros
6. Embajada de Japón - 461 metros
7. Embajada de Jordania - 362 metros
8. Embajada de Nueva Zelanda - 318 met

In [159]:
lat_center = (coordenadas[0][0] + coordenadas[1][0]) / 2
long_center = (coordenadas[0][1] + coordenadas[1][1]) / 2

start=(coordenadas[0][0], coordenadas[0][1])
finish=(coordenadas[1][0],coordenadas[1][1])

# Crear el mapa centrado en el punto medio de las coordenadas
mapa = folium.Map(location=[lat_center, long_center], zoom_start=14)

# Añadir la línea que conecta los puntos
folium.PolyLine(locations=[start, finish], color='blue').add_to(mapa)

# Mostrar el mapa
mapa

In [160]:
origen = (coordenadas[0][0], coordenadas[0][1]) 
destino = (coordenadas[1][0],coordenadas[1][1])
destino

(40.4313576, -3.6838303)

In [163]:

def abrir_ruta_bicicleta(origen_lat, origen_long, destino_lat, destino_long):
 
    url = f"https://www.google.com/maps/dir/{origen_lat},{origen_long}/{destino_lat},{destino_long}/data=!4m2!4m1!3e1"
     
    webbrowser.open(url)

origen_lat, origen_long = coordenadas[0][0], coordenadas[0][1]  
destino_lat, destino_long = coordenadas[1][0], coordenadas[1][1] 

abrir_ruta_bicicleta(origen_lat, origen_long, destino_lat, destino_long)